In [ ]:
%matplotlib inline


Created on Tue Feb 11 14:21:23 2020

@author: makinea1



In [ ]:
import sys
path = '/m/home/home8/80/makinea1/unix/pythonstuff/bfieldtools'
if path not in sys.path:
    sys.path.insert(0, path)


import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab
#import trimesh

from bfieldtools.mesh_class import Conductor
from bfieldtools.mesh_magnetics import magnetic_field_coupling
from bfieldtools.mesh_magnetics import magnetic_field_coupling_analytic
from bfieldtools.mesh_magnetics import scalar_potential_coupling
from bfieldtools.mesh_calculus import mass_matrix


from trimesh.creation import icosphere
#mesh = icosphere(3, 1)
#mesh2 = icosphere(3, 0.8)
import pkg_resources
from bfieldtools.mesh_calculus import gradient
file_obj = pkg_resources.resource_filename('bfieldtools',
                'example_meshes/cube.stl')
import trimesh
mesh = trimesh.load(file_obj, process=True)
mesh.vertices -= mesh.vertices.mean(axis=0)
mesh.apply_scale(0.2)

def Dmatrix(mesh1, mesh2, Nchunks=100):
    face_points = mesh2.vertices[mesh2.faces]
    weights = np.array([[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5]])
    # Combine vertices for quadrature points
    Rquad = np.einsum('...ij,ik->...kj', face_points, weights)
    R = Rquad.reshape(-1,3)
    U = scalar_potential_coupling(mesh1, R, Nchunks, multiply_coeff=False,
                                  approx_far=True)

    face_areas = mesh2.area_faces
    # Reshape and multiply by quadrature weights
    Dcomps = U.reshape(Rquad.shape[:2] + (len(mesh2.vertices),))*(face_areas[:, None, None]/3)
    # Sum up the quadrature points
    D = mesh2.faces_sparse @ Dcomps[:,0,:]
    D += mesh2.faces_sparse @ Dcomps[:,1,:]
    D += mesh2.faces_sparse @ Dcomps[:,2,:]


#    if mesh1 is mesh2:
    #%% Recalculate diagonals
    d = np.diag(D)
    D -= np.diag(d)
    # Make rows sum to -2*pi*(vertex area)
    d2 = -2*np.pi*mass_matrix(mesh2, lumped=False) - np.diag(D.sum(axis=1))
    D += d2
    # Make D solvable if it is a self coupling matrix
    D += np.ones_like(D)*np.max(np.linalg.svd(D, False, False))/D.shape[1]

    return D

def phi0x(r):
    return r[:,0]

def phi0y(r):
    return r[:,1]

def phi0z(r):
    return r[:,2]


def phi0_mat(mesh, func):
    face_points = mesh.vertices[mesh.faces]
    weights = np.array([[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5]])
    # Combine vertices for quadrature points
    Rquad = np.einsum('...ij,ik->...kj', face_points, weights)
    R = Rquad.reshape(-1,3)

    p = func(R)
    face_areas = mesh.area_faces
    # Reshape and multiply by quadrature weights
    pcomps = p.reshape(Rquad.shape[:2])*(face_areas[:, None]/3)
    # Sum up the quadrature points
    p_mat = mesh.faces_sparse @ pcomps[:,0]
    p_mat += mesh.faces_sparse @ pcomps[:,1]
    p_mat += mesh.faces_sparse @ pcomps[:,2]

    return p_mat

D = Dmatrix(mesh, mesh)
M = mass_matrix(mesh)
Ml = mass_matrix(mesh, lumped=True)


#%%
phi0_func = phi0x
pp = phi0_mat(mesh, phi0_func)
mu_r = 1000
c1 = (mu_r - 1)/(mu_r + 1)
u = np.linalg.solve(M + 1/(2*np.pi)*c1*D, 2*pp/(mu_r + 1))

m =mlab.triangular_mesh(*mesh.vertices.T, mesh.faces,
                        scalars=u, colormap='bwr')
m.actor.mapper.interpolate_scalars_before_mapping = True
m.module_manager.scalar_lut_manager.number_of_colors = 32
#%%
import pkg_resources
from bfieldtools.mesh_calculus import gradient
#Load simple plane mesh that is centered on the origin
file_obj = pkg_resources.resource_filename('bfieldtools',
                'example_meshes/10x10_plane_hires.obj')
import trimesh
plane = trimesh.load(file_obj, process=True)
t = np.eye(4)
t[1:3,1:3] = np.array([[0,1],[-1,0]])
plane.apply_transform(t)
plane.apply_scale(0.3)
plane = plane.subdivide()

#Uplane = scalar_potential_coupling(mesh, plane.vertices, multiply_coeff=False,
#                                   approx_far=True)
X,Y = np.meshgrid(np.linspace(-1.5,1.5,50), np.linspace(-1.5,1.5,50), indexing='ij')
pp = np.zeros((50*50,3))
pp[:, 0] = X.flatten()
pp[:, 1] = Y.flatten()


uprim = phi0_func(plane.vertices)
usec  = (mu_r - 1)/(4*np.pi)*Uplane@u
uplane = uprim - usec

Bplane = magnetic_field_coupling(mesh, plane.vertices)
bprim = pp*0
bprim[:,0] = 1
bplane = -(mu_r - 1)/(4*np.pi)*(1e7)*Bplane@u - bprim
#uplane[Uplane.sum(axis=1)]

#%%
mlab.figure()
m = mlab.triangular_mesh(*plane.vertices.T, plane.faces, scalars=uplane, colormap='bwr')
m.actor.mapper.interpolate_scalars_before_mapping = True
m.module_manager.scalar_lut_manager.number_of_colors = 32
#vectors = mlab.quiver3d(*plane.triangles_center.T, *(-gradient(uplane, plane)), color=(0,0,0),
#              scale_mode='none', scale_factor=0.1, mode='arrow')
#vectors.glyph.mask_input_points = True
#vectors.glyph.mask_points.random_mode_type = 1
#vectors.glyph.mask_points.on_ratio = 8
#vectors.glyph.glyph_source.glyph_position = 'center'

bplane[np.linalg.norm(bplane,axis=1)>5] =0
bvecs = mlab.quiver3d(*pp.T, *bplane.T,  color=(1,0,0),
              scale_mode='vector', scale_factor=0.02, mode='arrow')
bvecs.glyph.glyph_source.glyph_position = 'center'

bplane[:,2]=0
vecfield = mlab.pipeline.vector_field(*pp.T.reshape(3,50,50,1),
                                  *bplane.T.reshape(3,50,50,1))
vecnorm = mlab.pipeline.extract_vector_norm(vecfield)
streams = [] # create a list to hold all our streamlines (or flows if you speak MayaVi)

Nq=40
q = np.zeros((Nq, 3))
q[:,1] = np.linspace(-1.5, 1.5, Nq)
q[:,0] = 1.5
extent = np.array([-1.5, 1.5, -1.5, 1.5, 0, 0])
for qi in q: # for each charge, create a streamline seed
    stream = mlab.pipeline.streamline(vecnorm, seed_scale=0.01, seedtype='point',
                                      integration_direction='both',
                                      extent = np.array([-1.5, 1.5, -1.5, 1.5, 0, 0]),
                                      colormap='viridis') # the seed resolution is set to a minimum initially to avoid extra calculations
    stream.stream_tracer.initial_integration_step = 0.1 # the integration step for the runge kutta method
    stream.stream_tracer.maximum_propagation = 200.0 # the maximum length each step should reach - lowered to avoid messy output
    stream.seed.widget = stream.seed.widget_list[3]
    stream.seed.widget.position = qi # set the stream widget to the same position as the charge
#    stream.seed.widget.radius = dt * 2 # and its radius a bit bigger than the grid size
#    stream.seed.widget.theta_resolution = 30 # make the resolution high enough to give a fair number of lines
#    stream.seed.widget.phi_resolution = 1 # but we are looking at a plane for now, so let's not have any resolution in the z-direction
    stream.seed.widget.enabled = False # hide the widget itself
    streams.append(stream) # and eventually, add the stream to our list for convenience